# Predicción del precio de vehículos usados

Se va a construir un modelo de regresión lineal que pronostique el precio actual de vehículos usados a partir de algunas características. El marco de datos tiene las siguientes variables regresoras:

- `Car_Name`. Nombre del vehiculo.
- `Year`. Año de fabricación.
- `Selling_Price`. Precio de venta.
- `Driven_Kms`. Kilometraje recorrido.
- `Fuel_type`. Tipo de combustible.
- `Selling_Type`. Tipo de vendedor.
- `Transmission`. Tipo de transmisión.
- `Owner`. Número de propietarios.

Y la variable a predecir es:

- `Present_Price`. Precio actual.

In [1]:
#
# Carga de paquetes de interés
#

# Paquetes generales
import pandas as pd
pd.set_option('display.notebook_repr_html', False)

import os
import pickle
import gzip
import json

# Paquetes analíticos
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error

# 1. Preprocesamiento de datos

## 1.0. Lectura de los datos

In [2]:
def lectura_datos():
    """
    Lee los datos y retorna un marco de datos de pandas
    """

    direc_tr = "../files/input/train_data.csv.zip"
    direc_ts = "../files/input/test_data.csv.zip"

    df_tr = pd.read_csv(
        direc_tr,
        header=0,
        encoding='utf-8',
    )

    df_ts = pd.read_csv(
        direc_ts,
        header=0,
        encoding='utf-8',
    )

    return df_tr, df_ts

In [3]:
df_tr, df_ts = lectura_datos()

print(
    '===== DIMENSIÓN DEL MARCO DE DATOS =====',
    '=== Train ===',
    df_tr.shape,
    '',
    '',
    '=== Test ===',
    df_ts.shape,
    sep='\n'
)

===== DIMENSIÓN DEL MARCO DE DATOS =====
=== Train ===
(211, 9)


=== Test ===
(90, 9)


In [4]:
print(
    '===== TIPO DE DATOS =====',
    '=== Train ===',
    df_tr.dtypes,
    '',
    '',
    '=== Test ===',
    df_ts.dtypes,
    sep='\n'
)

===== TIPO DE DATOS =====
=== Train ===
Car_Name          object
Year               int64
Selling_Price    float64
Present_Price    float64
Driven_kms         int64
Fuel_Type         object
Selling_type      object
Transmission      object
Owner              int64
dtype: object


=== Test ===
Car_Name          object
Year               int64
Selling_Price    float64
Present_Price    float64
Driven_kms         int64
Fuel_Type         object
Selling_type      object
Transmission      object
Owner              int64
dtype: object


In [5]:
print(
    '===== VISUALIZACIÓN DE LAS ÚLTIMAS Y PRIMERAS TRES OBSERVACIONES =====',
    '=== Train ===',
    pd.concat([df_tr.head(3), df_tr.tail(3)], axis=0),
    '',
    '',
    '=== Test ===',
    pd.concat([df_ts.head(3), df_ts.tail(3)], axis=0),
    sep='\n'
)

===== VISUALIZACIÓN DE LAS ÚLTIMAS Y PRIMERAS TRES OBSERVACIONES =====
=== Train ===
               Car_Name  Year  Selling_Price  Present_Price  Driven_kms  \
0                  jazz  2016            7.4          8.500       15059   
1                   i10  2013            4.0          4.600       30000   
2    TVS Apache RTR 180  2011            0.5          0.826        6000   
208    Bajaj Pulsar 150  2006            0.1          0.750       92233   
209                 i20  2014            6.0          7.600       77632   
210                jazz  2016            6.0          8.400        4000   

    Fuel_Type Selling_type Transmission  Owner  
0      Petrol       Dealer    Automatic      0  
1      Petrol       Dealer       Manual      0  
2      Petrol   Individual       Manual      0  
208    Petrol   Individual       Manual      0  
209    Diesel       Dealer       Manual      0  
210    Petrol       Dealer       Manual      0  


=== Test ===
   Car_Name  Year  Selling_Pric

In [6]:
print(
    '===== DESCRIPCIÓN DE LAS COLUMNAS =====',
    '=== Train ===',
    df_tr.describe(),
    '',
    '',
    '=== Test ===',
    df_ts.describe(),
    sep='\n'
)

===== DESCRIPCIÓN DE LAS COLUMNAS =====
=== Train ===
              Year  Selling_Price  Present_Price     Driven_kms       Owner
count   211.000000     211.000000     211.000000     211.000000  211.000000
mean   2013.644550       4.692512       7.561090   35578.009479    0.047393
std       2.794843       4.819333       7.382453   28912.475577    0.271907
min    2003.000000       0.100000       0.480000    1200.000000    0.000000
25%    2012.000000       1.025000       1.365000   15000.000000    0.000000
50%    2014.000000       3.750000       6.400000   32000.000000    0.000000
75%    2016.000000       6.050000       9.900000   47500.000000    0.000000
max    2018.000000      23.500000      35.960000  213000.000000    3.000000


=== Test ===
              Year  Selling_Price  Present_Price     Driven_kms      Owner
count    90.000000      90.000000      90.000000      90.000000  90.000000
mean   2013.588889       4.588111       7.786444   40157.211111   0.033333
std       3.122569    

## 1.1. Creación de la columna `Age`

In [7]:
#
# Función para el cálculo de la edad del vehículo si estamos en 2021
def columna_age(df):
    df['Age'] = df['Year'].apply(
        lambda x: 2021 - x
    )
    
    return df

#
# Aplicación de la función
#
df_tr = columna_age(df_tr)
df_ts = columna_age(df_ts)

#
# Verificación
#
print(
    '===== VALIDACIÓN DE LA COLUMNA Age =====',
    '=== Train ===',
    df_tr[['Age', 'Year']].head(),
    '',
    '',
    '=== Test ===',
    df_ts[['Age', 'Year']].head(),
    sep='\n'
)


===== VALIDACIÓN DE LA COLUMNA Age =====
=== Train ===
   Age  Year
0    5  2016
1    8  2013
2   10  2011
3    5  2016
4    8  2013


=== Test ===
   Age  Year
0    8  2013
1    4  2017
2   10  2011
3    6  2015
4    6  2015


In [8]:
#
# Estadísticas para la nueva columna
#
print(
    '===== ESTADÍSTICAS DE LA COLUMNA Age =====',
    '=== Train ===',
    df_tr[['Age',]].describe(),
    '',
    '',
    '=== Test ===',
    df_ts[['Age',]].describe(),
    sep='\n'
)

===== ESTADÍSTICAS DE LA COLUMNA Age =====
=== Train ===
              Age
count  211.000000
mean     7.355450
std      2.794843
min      3.000000
25%      5.000000
50%      7.000000
75%      9.000000
max     18.000000


=== Test ===
             Age
count  90.000000
mean    7.411111
std     3.122569
min     4.000000
25%     5.000000
50%     7.000000
75%     9.000000
max    18.000000


## 1.2. Eliminación de las columnas `Year` y `Car_Name`

In [9]:
#
# Eliminación de las columnas Year y Car_Name
#

def eliminacion_year_carname(df: pd.DataFrame):
    df.drop(
        ['Year', 'Car_Name'],
        axis=1,
        inplace=True,
    )

#
# Aplicación de la eliminación
#
eliminacion_year_carname(df_tr)
eliminacion_year_carname(df_ts)

#
# Validación
#
print(
    '===== NOMBRE DE LAS COLUMNAS DEL MARCO DE DATOS =====',
    '=== Train ===',
    df_tr.columns,
    '',
    '',
    '=== Test ===',
    df_ts.columns,
    sep='\n'
)

===== NOMBRE DE LAS COLUMNAS DEL MARCO DE DATOS =====
=== Train ===
Index(['Selling_Price', 'Present_Price', 'Driven_kms', 'Fuel_Type',
       'Selling_type', 'Transmission', 'Owner', 'Age'],
      dtype='object')


=== Test ===
Index(['Selling_Price', 'Present_Price', 'Driven_kms', 'Fuel_Type',
       'Selling_type', 'Transmission', 'Owner', 'Age'],
      dtype='object')


## 2. División en conjuntos de entrenamiento y validación

En este caso, como ya están dados los conjuntos de entrenamiento y validación, basta con hacer la separación de la variable respuesta de las regresoras.

In [10]:
#
# Separación de variable respuesta de la matriz de diseño
#

def separacion_variables(df: pd.DataFrame):
    df_aux = df.copy()

    y = df_aux.pop('Present_Price')

    return y, df_aux

# Para el conjunto de entrenamiento
y_tr, X_tr = separacion_variables(df_tr)

# Para el conjunto de validación
y_ts, X_ts = separacion_variables(df_ts)

#
# Verificación
#
print(
    '===== SEPARACIÓN DE LA MATRIZ DE DISEÑO Y LA VARIABLE RESPUESTA =====',
    '=== Train ===',
    X_tr.columns,
    y_tr.shape,
    '',
    '',
    '=== Test ===',
    X_ts.columns,
    y_ts.shape,
    sep='\n'
)

===== SEPARACIÓN DE LA MATRIZ DE DISEÑO Y LA VARIABLE RESPUESTA =====
=== Train ===
Index(['Selling_Price', 'Driven_kms', 'Fuel_Type', 'Selling_type',
       'Transmission', 'Owner', 'Age'],
      dtype='object')
(211,)


=== Test ===
Index(['Selling_Price', 'Driven_kms', 'Fuel_Type', 'Selling_type',
       'Transmission', 'Owner', 'Age'],
      dtype='object')
(90,)


# 3. Pipeline

Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:
- Transforma las variables categoricas usando el método *one-hot-encoding*.
- Escala las variables numéricas al intervalo [0, 1].
- Selecciona las K mejores entradas.
- Ajusta un modelo de regresion lineal.

In [11]:
X_tr.dtypes

Selling_Price    float64
Driven_kms         int64
Fuel_Type         object
Selling_type      object
Transmission      object
Owner              int64
Age                int64
dtype: object

In [12]:
#
# Separación de variables categóricas y cuantitativas
#

categoricas = ['Fuel_Type', 'Selling_type', 'Transmission']
cuantitativas = [x for x in X_tr.columns if x not in categoricas]

#
# Preprocesador para las columnas de entrada
#
preprocesador = ColumnTransformer(
    transformers=[
        ('categoricas', OneHotEncoder(), categoricas),
        ('cuantitativas', MinMaxScaler(), cuantitativas),
    ]
)

#
# Pipeline
#
pipeline = Pipeline(
    [
        ('columnas', preprocesador),
        ('mejores_k', SelectKBest(
            score_func=f_regression,
        )),
        ('regresor', LinearRegression()),
    ]
)

pipeline

Pipeline(steps=[('columnas',
                 ColumnTransformer(transformers=[('categoricas',
                                                  OneHotEncoder(),
                                                  ['Fuel_Type', 'Selling_type',
                                                   'Transmission']),
                                                 ('cuantitativas',
                                                  MinMaxScaler(),
                                                  ['Selling_Price',
                                                   'Driven_kms', 'Owner',
                                                   'Age'])])),
                ('mejores_k',
                 SelectKBest(score_func=<function f_regression at 0x1686a4ee0>)),
                ('regresor', LinearRegression())])

# 4. Optimización de hiperparámetros

In [13]:
# Definir proporciones de búsqueda del número de columnas
proporciones = [0.2, 0.5, 0.8, 1.0]

preprocesador.fit(X_tr)
num_columnas = preprocesador.transform(X_tr).shape[1]

# Número de columnas a buscar
valores_k = [max(1, int(p * num_columnas)) for p in proporciones]

In [14]:
#
# Grilla de hiperparámetros
#
param_grid = {
    'mejores_k__k': valores_k,  # Número de características seleccionadas
    'regresor__fit_intercept': [True, False],  # Ajuste del intercepto
    'columnas__categoricas__drop': ['first', None]  # Cómo manejar la codificación OneHot
}

#
# Definición del modelo con GridSearchCV
#
model = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=10,
    n_jobs=10,
)

In [15]:
#
# Ajuste del modelo
#
model.fit(X_tr, y_tr)

/Users/scuartasr/Documents/Maestría/Analitica/Clase/Labs/2024-2-LAB-05-prediccion-de-precios-usando-linreg-scuartasr/.venv/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:779: UserWarning: k=11 is greater than n_features=8. All the features will be returned.
  warnings.warn(
/Users/scuartasr/Documents/Maestría/Analitica/Clase/Labs/2024-2-LAB-05-prediccion-de-precios-usando-linreg-scuartasr/.venv/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:779: UserWarning: k=11 is greater than n_features=8. All the features will be returned.
  warnings.warn(
/Users/scuartasr/Documents/Maestría/Analitica/Clase/Labs/2024-2-LAB-05-prediccion-de-precios-usando-linreg-scuartasr/.venv/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:779: UserWarning: k=11 is greater than n_features=8. All the features will be returned.
  warnings.warn(
/Users/scuartasr/Documents/Maestría/Analitica/Clase/Labs/2024-2-LAB-05-pred

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('columnas',
                                        ColumnTransformer(transformers=[('categoricas',
                                                                         OneHotEncoder(),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission']),
                                                                        ('cuantitativas',
                                                                         MinMaxScaler(),
                                                                         ['Selling_Price',
                                                                          'Driven_kms',
                                                                          'Owner',
                                                                          'Age'])])),
                                       ('mejores_k',
                                        SelectKBest(score_func=<function f_regression at 0x1686a4ee0>)),
                                       ('regresor', LinearRegression())]),
             n_jobs=10,
             param_grid={'columnas__categoricas__drop': ['first', None],
                         'mejores_k__k': [2, 5, 8, 11],
                         'regresor__fit_intercept': [True, False]},
             scoring='neg_mean_squared_error')

In [16]:
model.best_params_

{'columnas__categoricas__drop': 'first',
 'mejores_k__k': 8,
 'regresor__fit_intercept': False}

In [17]:
print(
    "===== MÉTRICAS =====",
    "Train:",
    mean_squared_error(y_tr, model.predict(X_tr)),
    "",
    "------",
    "Test:",
    mean_squared_error(y_ts, model.predict(X_ts)),
    sep='\n',
)

===== MÉTRICAS =====
Train:
5.881687967089613

------
Test:
32.311461797621355


# 5. Guardado del modelo comprimido

In [18]:
# Carpeta donde se guarda el modelo
direccion = '../files/models'
os.makedirs(direccion, exist_ok=True)

# Guardado del modelo
direc_modelo = '../files/models/model.pkl.gz'
with gzip.open(direc_modelo, 'wb') as file:
    pickle.dump(model, file)

# 6. Cálculo de métricas

In [19]:
def calculo_metricas(
        modelo, y_tr, y_ts, X_tr, X_ts, directorio, nombre
):
    # Predicción
    y_tr_pred = modelo.predict(X_tr)
    y_ts_pred = modelo.predict(X_ts)

    # Métricas para entrenamiento
    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'r2': float(r2_score(y_tr, y_tr_pred)),
        'mse': float(mean_squared_error(y_tr, y_tr_pred)),
        'mad': float(median_absolute_error(y_tr, y_tr_pred)),
    }

    # Métricas para validcion
    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'r2': float(r2_score(y_ts, y_ts_pred)),
        'mse': float(mean_squared_error(y_ts, y_ts_pred)),
        'mad': float(median_absolute_error(y_ts, y_ts_pred)),
    }

    # Almacenamiento
    os.makedirs(directorio, exist_ok=True)

    # Guardado de métricas
    with open(os.path.join(directorio, nombre), 'w') as file:
        file.write(json.dumps(metrics_train) + '\n')
        file.write(json.dumps(metrics_test) + '\n')

In [20]:
calculo_metricas(
    modelo=model,
    y_tr=y_tr,
    y_ts=y_ts,
    X_tr=X_tr,
    X_ts=X_ts,
    directorio='../files/output',
    nombre='metrics.json',
)

In [21]:
X_tr

     Selling_Price  Driven_kms Fuel_Type Selling_type Transmission  Owner  Age
0             7.40       15059    Petrol       Dealer    Automatic      0    5
1             4.00       30000    Petrol       Dealer       Manual      0    8
2             0.50        6000    Petrol   Individual       Manual      0   10
3             3.15       15000    Petrol       Dealer       Manual      0    5
4             1.25       15000    Petrol   Individual       Manual      0    8
..             ...         ...       ...          ...          ...    ...  ...
206           2.55       57000    Petrol       Dealer       Manual      0   10
207           3.95       36000    Petrol       Dealer       Manual      0    6
208           0.10       92233    Petrol   Individual       Manual      0   15
209           6.00       77632    Diesel       Dealer       Manual      0    7
210           6.00        4000    Petrol       Dealer       Manual      0    5

[211 rows x 7 columns]